<a href="https://colab.research.google.com/github/muzakkialfarisi/TUGAS_AKHIR/blob/main/Klasifikasi_Multi_label_pada_Hadis_Sahih_Bukhari_Terjemahan_Bahasa_Indonesia_Menggunakan_Convolutional_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Klasifikasi Multi-label pada Hadis Sahih Bukhari Terjemahan Bahasa Indonesia Menggunakan Convolutional Neural Networks**

muzakkialfarisi@student.telkomuniversity.ac.id

widiwdu@telkomuniversity.ac.id

adiwijaya@telkomuniversity.ac.id

In [ ]:
from google.colab import drive
drive.mount('drive/')
import pandas as pd
import csv
import re
import string
import time
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.metrics import *
from sklearn.metrics import hamming_loss

Drive already mounted at drive/; to attempt to forcibly remount, call drive.mount("drive/", force_remount=True).


**Read Data Train**

In [ ]:
train = pd.read_csv("drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Dataset/hadits2k.csv", names=["hadis"])
train.head()

,hadis
0,"Semua perbuatan tergantung niatnya, dan (balas..."
1,Terkadang datang kepadaku seperti suara gemeri...
2,Bacalah? Beliau menjawab: Aku tidak bisa baca....
3,sangat kuat keinginannya untuk menghafalkan ap...
4,manusia yang paling lembut terutama pada bulan...


In [ ]:
label = pd.read_csv("drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Dataset/label2k.csv")
label.head()

,anjuran,larangan,informasi
0,0,0,1
1,0,0,1
2,0,0,1
3,0,1,1
4,0,0,1


**Remove Punctuation**

In [ ]:
punc = string.punctuation.replace('<', '').replace('>', '')
pat = re.compile(f'[{punc}]')
train["h_punc"] = train['hadis'].replace(pat, '')

**Case Folding**

In [ ]:
train['h_punc_low'] = train['h_punc'].str.lower()

**Stemming**

In [ ]:
pip install Sastrawi

     |████████████████████████████████| 209 kB 8.0 MB/s 


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [ ]:
stemmer = StemmerFactory().create_stemmer()

list_low = train['h_punc_low'].tolist()

text_stemmed = map(lambda x: stemmer.stem(x), list_low)

list_stemmed = []
for i in text_stemmed:
  list_stemmed.append(i)

train['h_punc_low_stem'] = pd.DataFrame(list_stemmed)

In [ ]:
train.head()

,hadis,h_punc,h_punc_low,h_punc_low_stem
0,"Semua perbuatan tergantung niatnya, dan (balas...",Semua perbuatan tergantung niatnya dan balasan...,semua perbuatan tergantung niatnya dan balasan...,semua buat gantung niat dan balas bagi tiaptia...
1,Terkadang datang kepadaku seperti suara gemeri...,Terkadang datang kepadaku seperti suara gemeri...,terkadang datang kepadaku seperti suara gemeri...,terkadang datang kepada seperti suara gerincin...
2,Bacalah? Beliau menjawab: Aku tidak bisa baca....,Bacalah Beliau menjawab Aku tidak bisa baca Na...,bacalah beliau menjawab aku tidak bisa baca na...,baca beliau jawab aku tidak bisa baca nabi jel...
3,sangat kuat keinginannya untuk menghafalkan ap...,sangat kuat keinginannya untuk menghafalkan ap...,sangat kuat keinginannya untuk menghafalkan ap...,sangat kuat ingin untuk hafal apa yang turun a...
4,manusia yang paling lembut terutama pada bulan...,manusia yang paling lembut terutama pada bulan...,manusia yang paling lembut terutama pada bulan...,manusia yang paling lembut utama pada bulan ra...


In [ ]:
def tokenizing(dataTrain):
  tokenizer = Tokenizer(num_words=732)
  tokenizer.fit_on_texts(list(dataTrain))
  dataTrain = tokenizer.texts_to_sequences(dataTrain)
  dataTrain = sequence.pad_sequences(dataTrain, maxlen=100)
  return dataTrain

In [ ]:
def get_cnn1_model():
  opt = tf.keras.optimizers.Adam(learning_rate=0.01)
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=2000, input_length=732, output_dim=100))
  model.add(tf.keras.layers.Conv1D(filters=512, kernel_size=3, padding='valid', strides=1, activation='relu'))
  model.add(tf.keras.layers.GlobalMaxPool1D())
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(3, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
def get_cnn2_model():
  opt = tf.keras.optimizers.Adam(learning_rate=0.01)
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=2000, input_length=732, output_dim=100))
  model.add(tf.keras.layers.Conv1D(filters=512, kernel_size=3, padding='same', strides=1, activation='relu'))
  model.add(tf.keras.layers.GlobalMaxPool1D())
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(3, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
train2 = tokenizing(train['h_punc_low_stem'].values)
X_train2, X_test2, y_train2, y_test2 = train_test_split(train2, label, test_size=0.2, random_state=3)

train3 = tokenizing(train['h_punc_low'].values)
X_train3, X_test3, y_train3, y_test3 = train_test_split(train3, label, test_size=0.2, random_state=3)

**CNN tanpa padding**

In [ ]:
model_cnn_valid_2 = get_cnn1_model()
time_cnn_valid_2 = time.time()
model_cnn_valid_fit_2 = model_cnn_valid_2.fit(X_train2, y_train2, epochs=15, validation_data=(X_test2, y_test2), verbose=False, batch_size=32)
time_cnn_valid_2 = time.time() - time_cnn_valid_2
# model_cnn_valid_2.save('drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Hasil Uji/model_cnn_valid_1')

In [ ]:
model_cnn_valid_3 = get_cnn1_model()
time_cnn_valid_3 = time.time()
model_cnn_valid_fit_3 = model_cnn_valid_3.fit(X_train3, y_train3, epochs=15, validation_data=(X_test3, y_test3), verbose=False, batch_size=32)
time_cnn_valid_3 = time.time() - time_cnn_valid_3
# model_cnn_valid_3.save('drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Hasil Uji/model_cnn_valid_2')

**LOAD MODEL TERSIMPAN VALID**

In [ ]:
model_cnn_11 = keras.models.load_model('drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Hasil Uji/model_cnn_valid_1')

y_pred_cnn_11 = model_cnn_11.predict(X_test2, verbose=False)
prediction_cnn_11 = (y_pred_cnn_11 > 0.5)

val1 = []
val1.append(round(hamming_loss(y_test2, prediction_cnn_11), 4))
val1.append(round(time_cnn_valid_2, 4))

In [ ]:
model_cnn_12 = keras.models.load_model('drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Hasil Uji/model_cnn_valid_2')

y_pred_cnn_12 = model_cnn_12.predict(X_test3, verbose=False)
prediction_cnn_12 = (y_pred_cnn_12 > 0.5)

val1.append(round(hamming_loss(y_test3, prediction_cnn_12), 4))
val1.append(round(time_cnn_valid_3, 4))

**CNN dengan padding**

In [ ]:
model_cnn_same_2 = get_cnn2_model()
time_cnn_same_2 = time.time()
model_cnn_same_fit_2 = model_cnn_same_2.fit(X_train2, y_train2, epochs=15, validation_data=(X_test2, y_test2), verbose=False, batch_size=32)
time_cnn_same_2 = time.time() - time_cnn_same_2
# model_cnn_same_2.save('drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Hasil Uji/model_cnn_same_1')

In [ ]:
model_cnn_same_3 = get_cnn2_model()
time_cnn_same_3 = time.time()
model_cnn_same_fit_3 = model_cnn_same_3.fit(X_train3, y_train3, epochs=15, validation_data=(X_test3, y_test3), verbose=False, batch_size=32)
time_cnn_same_3 = time.time() - time_cnn_same_3
# model_cnn_same_3.save('drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Hasil Uji/model_cnn_same_2')

**`LOAD MODEL TERSIMPAN SAME`**


In [ ]:
model_cnn_21 = keras.models.load_model('drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Hasil Uji/model_cnn_same_1')

y_pred_cnn_21 = model_cnn_21.predict(X_test2, verbose=False)
prediction_cnn_21 = (y_pred_cnn_21 > 0.5)

val2 = []
val2.append(round(hamming_loss(y_test2, prediction_cnn_21), 4))
val2.append(round(time_cnn_same_2, 4))

In [ ]:
model_cnn_22 = keras.models.load_model('drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Hasil Uji/model_cnn_same_2')

y_pred_cnn_22 = model_cnn_22.predict(X_test3, verbose=False)
prediction_cnn_22 = (y_pred_cnn_22 > 0.5)

val2.append(round(hamming_loss(y_test3, prediction_cnn_22), 4))
val2.append(round(time_cnn_same_3, 4))

In [ ]:
print("                   dengan stemming - tanpa stemming")
print("tanpa  padding : ",val1)
print("dengan padding : ",val2)

                   dengan stemming - tanpa stemming
tanpa  padding :  [0.0567, 55.1915, 0.055, 55.5303]
dengan padding :  [0.0575, 56.5767, 0.0567, 56.8204]


**RNNS**

In [ ]:
def get_rnns_model():
  opt = tf.keras.optimizers.Adam(learning_rate=0.01)
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=2000, input_length=732, output_dim=100))
  model.add(tf.keras.layers.SimpleRNN(512, return_sequences=True))
  model.add(tf.keras.layers.GlobalMaxPool1D())
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(3, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
model_rnns = get_rnns_model()
time_rnns = time.time()
model_fit_rnns = model_rnns.fit(X_train3, y_train3, epochs=15, validation_data=(X_test3, y_test3), verbose=False, batch_size=32)
time_rnns = time.time() - time_rnns
# model_rnns.save('drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Hasil Uji/model_rnns')

**LOAD MODEL RNN**

In [ ]:
load_rnns = keras.models.load_model('drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Hasil Uji/model_rnns')

y_pred_load_rnns = load_rnns.predict(X_test3, verbose=False)
prediction_load_rnns = (y_pred_load_rnns > 0.5)

load_rnns = []
load_rnns.append(round(hamming_loss(y_test3, prediction_load_rnns), 4))
load_rnns.append(round(time_rnns, 4))

In [ ]:
print(load_rnns)

[0.1, 197.9882]


**LSTM**

In [ ]:
def get_lstm_model():
  opt = tf.keras.optimizers.Adam(learning_rate=0.01)
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim=2000, input_length=732, output_dim=100))
  model.add(tf.keras.layers.LSTM(512, return_sequences=True))
  model.add(tf.keras.layers.GlobalMaxPool1D())
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(3, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['categorical_accuracy'])
  return model

In [ ]:
model_lstm = get_lstm_model()
time_lstm = time.time()
model_fit_lstm = model_lstm.fit(X_train3, y_train3, epochs=15, validation_data=(X_test3, y_test3), verbose=False, batch_size=32)
time_lstm = time.time() - time_lstm
# model_lstm.save('drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Hasil Uji/model_lstm')

**LOAD MODEL LSTM**

In [ ]:
load_lstm = keras.models.load_model('drive/My Drive/Colab Notebooks/Tugas Akhir/ADD/Hasil Uji/model_lstm')

y_pred_load_lstm = load_lstm.predict(X_test3, verbose=False)
prediction_load_lstm = (y_pred_load_lstm > 0.5)

load_lstm = []
load_lstm.append(round(hamming_loss(y_test3, prediction_load_lstm), 4))
load_lstm.append(round(time_lstm, 4))

In [ ]:
print(load_lstm)

[0.075, 890.8086]
